In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/sentiment_analysis.zip", 'r')
zip_ref.extractall("/content/files/")
zip_ref.close()

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification,get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

In [ ]:
import pandas as pd

df=pd.read_csv("/content/files/train.csv",encoding="iso-8859-1")
df.head()

In [ ]:
import re

def data_preprocessing(txt):
    regex = r'[^\w\s]|[\U0001f600-\U0001f64f\U0001f300-\U0001f5ff\U0001f680-\U0001f6ff\U0001f1e0-\U0001f1ff]'
    txt=re.sub(regex," ",txt)
    txt=re.sub("\.|\,|\/|\-"," ",txt) 
    txt=re.sub("\s*\s"," ",txt)
    return text

In [ ]:
for i in range(len(df)):
    df.loc[i,"txt"]=data_preprocessing(str(df.loc[i,"txt"]))

In [ ]:
df["sentiment"]=df["sentiment"].replace({"negative":0,"neutral":1,"positive":2})

In [ ]:
df=df[["txt","sentiment"]]

In [ ]:
from sklearn.model_selection import train_test_split 

train_txt,test_txt,train_lbl,test_lbl=train_test_split(df["txt"],df["sentiment"],test_size=0.5)
valid_txt,test_txt,valid_lbl,test_lbl=train_test_split(test_txt,test_lbl,test_size=0.5)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
#import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
     
    

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
classes = 3

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
     

In [ ]:
def convert_example_to_feature(review):
    return bert_tokenizer.encode_plus(review, add_special_tokens = True, max_length = 512, padding='max_length',
                truncation=True, return_attention_mask = True)

In [ ]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {"input_ids": input_ids, "token_type_ids": token_type_ids, "attention_mask": attention_masks,}, label

In [ ]:
def encode_examples(ds):
  
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    for review, label in ds:
        bert_input = convert_example_to_feature(review)
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])

    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)


In [ ]:
batch_size = 8

ds_train = zip(train_txt, train_lbl)
ds_test = zip(test_txt, test_lbl)
ds_train_encoded = encode_examples(ds_train).shuffle(len(train_txt)).batch(batch_size)
ds_test_encoded = encode_examples(ds_test).batch(batch_size)
     

In [ ]:
model_save_path = 'bert_model.h5'

path = "/content/"

## Initialize pre-built BERT-based classifier from transformers
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

bert_model.summary()

In [ ]:
learning_rate = 2e-5


number_of_epochs = 5

# choosing Adam optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-08)



loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

bert_model.compile(loss=loss, optimizer=optimizer, metrics=metric)

In [ ]:
past = bert_model.fit(ds_train_encoded, batch_size=batch_size, epochs=number_of_epochs, validation_data=ds_test_encoded)

In [ ]:
bert_model.save_weights(model_save_path)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

matplotlib.rcParams['figure.dpi'] = 150

In [ ]:
def plot1(past):
    acc = past.past['accuracy']
    val_acc = past.past['val_accuracy']
    loss = past.past['loss']
    val_loss = past.past['val_loss']

    epochs = range(1, len(acc) + 1)
    ## Accuracy plot
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    ## Loss plot
    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

In [ ]:
def plot2(past):
    pd.DataFrame(past.past).plot(figsize=(8, 5))
    plt.grid(True)
    plt.show()

    
    
plot2(past)


In [ ]:
bert_model.evaluate(ds_test_encoded, batch_size=batch_size)

In [ ]:
y_test_pred = bert_model.predict(ds_test_encoded, batch_size=batch_size)
     

In [ ]:
y_test_pred_class = y_test_pred[0].argmax(axis=1)